In [1]:
import FraudDetectionModelDataPrep as dataPrep

In [2]:
dir(dataPrep)

['LabelEncoder',
 'LogisticRegression',
 'StratifiedKFold',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'classification_report',
 'confusion_matrix',
 'dataPrepAutomation',
 'np',
 'pd',
 'plt',
 'smoteBalancing',
 'sns',
 'train_test_split']

In [3]:
inputDF = dataPrep.pd.read_excel(r"D:\Learnerea\Tables\to_test_the_model.xlsx")

In [5]:
inputDF.shape

(4500, 11)

In [6]:
features, target = dataPrep.dataPrepAutomation(inputDF)

In [9]:
features.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER
0,303,13363860.78,0.00,0.00,13656437.62,27020298.40,0,False,False,False,False,True
1,355,340375.64,9758.51,0.00,4764447.25,5104822.89,0,False,False,False,False,True
2,404,276737.58,0.00,0.00,962518.44,1239256.03,0,False,True,False,False,False
3,258,314027.77,650090.00,336062.23,0.00,314027.77,0,False,True,False,False,False
4,36,33468.45,0.00,0.00,149332.38,182800.83,0,False,True,False,False,False


In [10]:
import pickle

In [11]:
with open("ensemble_model.pkl","rb") as file:
    ensemble_model = pickle.load(file)

In [14]:
prediction = ensemble_model.predict(features)
prediction

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [17]:
print(dataPrep.classification_report(target,prediction))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4000
           1       0.91      1.00      0.95       500

    accuracy                           0.99      4500
   macro avg       0.96      0.99      0.97      4500
weighted avg       0.99      0.99      0.99      4500



In [18]:
with open("smote_logistic.pkl","rb") as file:
    smote_logistic = pickle.load(file)

In [19]:
smoteLogPred = smote_logistic.predict(features)
smoteLogPred

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [20]:
print(dataPrep.classification_report(target,smoteLogPred))

              precision    recall  f1-score   support

           0       0.99      0.92      0.95      4000
           1       0.58      0.90      0.71       500

    accuracy                           0.92      4500
   macro avg       0.79      0.91      0.83      4500
weighted avg       0.94      0.92      0.92      4500



In [21]:
smoteLogProb = smote_logistic.predict_proba(features)

In [24]:
smoteLogProbdata = smoteLogProb[:,1]

In [27]:
finalData = dataPrep.pd.DataFrame(zip(target,prediction,smoteLogPred,smoteLogProbdata),columns=['isFraud','ensem_pred','smote_pred','smote_prob'])

In [28]:
finalOutcome = dataPrep.pd.concat([features,finalData],axis=1)
finalOutcome.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,isFraud,ensem_pred,smote_pred,smote_prob
0,303,13363860.78,0.00,0.00,13656437.62,27020298.40,0,False,False,False,False,True,0,0,0,4.550288e-79
1,355,340375.64,9758.51,0.00,4764447.25,5104822.89,0,False,False,False,False,True,0,0,0,4.131617e-03
2,404,276737.58,0.00,0.00,962518.44,1239256.03,0,False,True,False,False,False,0,0,0,8.529435e-03
3,258,314027.77,650090.00,336062.23,0.00,314027.77,0,False,True,False,False,False,0,0,1,9.799595e-01
4,36,33468.45,0.00,0.00,149332.38,182800.83,0,False,True,False,False,False,0,0,0,3.666744e-01


In [29]:
finalOutcome.to_excel(r"D:\Learnerea\Tables\dataforAnalysis.xlsx")